sss 11.02 xgb 10.50
전 시간대의 PM 데이터를 변수로 추가.

!pip install xgboost  #함수 install 다 추가할 것

In [1]:
import time
import re
import datetime as dt
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import sin,cos,arccos,pi,round
import math
import warnings
warnings.filterwarnings('ignore')

## 지역 측정
### 지역의 위치에 대해서 측정

In [2]:
loc_PM = pd.read_csv('./META/pmmap.csv', encoding = 'UTF-8',error_bad_lines=False)
loc_PM.head()

,Location,Latitude,Longitude,Description
0,아름동,36.512252,127.246789,세종특별자치시 보듬3로 114 아름동커뮤니티센터 옥상 (아름동)
1,신흥동,36.592887,127.292550,세종 조치원읍 군청로 87-16(신흥동) 세종특별자치시 조치원청사 옥상
2,노은동,36.368242,127.318498,대전 유성구 노은동로 87번길 89(노은1동 주민센터) 노은1동 주민센터 3층 옥상
3,문창동,36.317215,127.437825,대전 중구 보문로 20번길 38(문창동 주민센터) 문창동 주민센터
4,읍내동,36.372388,127.417714,대전 대덕구 대전로 1331번길 75(태아산업(주)) 태아산업(주)


In [3]:
loc_PM.sort_values(by=['Location'], ascending=True,inplace=True)
loc_PM.set_index('Location',inplace=True)

In [4]:
#Description Drop -> 스크린 공간을 너무 많이 잡아 먹어서 삭제 
loc_PM.drop('Description',axis = 1,inplace = True)

In [5]:
loc_PM.head()

,Latitude,Longitude
Location,,
공주,36.446951,127.119209
노은동,36.368242,127.318498
논산,36.199217,127.087021
대천2동,36.353148,126.589735
독곶리,36.987579,126.391672


In [6]:
loc_AWS = pd.read_csv('./META/awsmap.csv', index_col = 'Location', encoding = 'UTF-8',error_bad_lines=False)
loc_AWS.head()

,Latitude,Longitude,Description
Location,,,
오월드,36.2913,127.3959,대전광역시 중구사정공원로 70 오월드 내 플라워랜드
세천,36.3402,127.4938,대전광역시 동구세천동 63-1
장동,36.4135,127.4382,대전광역시 대덕구장동 360-2
세종고운,36.5315,127.2406,세종특별자치시 고운동산 25번지
세종금남,36.4585,127.2688,세종특별자치시 시군구금남면 성덕리 77-3


In [7]:
#Description Drop -> 스크린 공간을 너무 많이 잡아 먹어서 삭제 
loc_AWS.drop('Description',axis = 1,inplace = True)

In [8]:
loc_AWS.head()

,Latitude,Longitude
Location,,
오월드,36.2913,127.3959
세천,36.3402,127.4938
장동,36.4135,127.4382
세종고운,36.5315,127.2406
세종금남,36.4585,127.2688


In [9]:
# Train data import
PM_loc = []

for nm_PM in loc_PM.index:
    pm = pd.read_csv(f'./TRAIN/{nm_PM}.csv',encoding = 'UTF-8',error_bad_lines=False)
    PM_loc.append(pm)
    globals()['PM_{}'.format(nm_PM)] = pm
    
for nm_AWS in loc_AWS.index:
    globals()['AWS_{}'.format(nm_AWS)]=pd.read_csv(f'./TRAIN_AWS/{nm_AWS}.csv',encoding = 'UTF-8',error_bad_lines=False)

In [10]:
# Test data import
TPM_loc = []

for nm_PM in loc_PM.index:
    tpm = pd.read_csv(f'./TEST_INPUT/{nm_PM}.csv',encoding = 'UTF-8',error_bad_lines=False)
    TPM_loc.append(tpm)
    globals()['TPM_{}'.format(nm_PM)] = tpm
    
for nm_AWS in loc_AWS.index:
    globals()['TAWS_{}'.format(nm_AWS)]=pd.read_csv(f'./TEST_AWS/{nm_AWS}.csv',encoding = 'UTF-8',error_bad_lines=False)

In [11]:
AWS_공주

,연도,일시,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0,01-01 00:00,공주,0.173776,0.201944,0.023018,0.0,0.828
1,0,01-01 01:00,공주,0.176935,0.168611,0.030691,0.0,0.831
2,0,01-01 02:00,공주,0.180095,0.087222,0.033248,0.0,0.784
3,0,01-01 03:00,공주,0.178515,0.087222,0.025575,0.0,0.745
4,0,01-01 04:00,공주,0.164297,0.113889,0.020460,0.0,0.750
...,...,...,...,...,...,...,...,...
35059,3,12-31 19:00,공주,0.287520,0.765833,0.071611,0.0,0.577
35060,3,12-31 20:00,공주,0.281201,0.753056,0.063939,0.0,0.586
35061,3,12-31 21:00,공주,0.260664,0.000000,0.010230,0.0,0.674
35062,3,12-31 22:00,공주,0.262243,0.011944,0.020460,0.0,0.660


In [12]:
TAWS_공주

,연도,일시,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,4,01-01 00:00,공주,0.244866,0.123333,0.038363,0.0,0.647
1,4,01-01 01:00,공주,0.232227,0.167778,0.033248,0.0,0.648
2,4,01-01 02:00,공주,0.206951,0.000000,0.002558,0.0,0.734
3,4,01-01 03:00,공주,0.199052,0.000000,0.002558,0.0,0.753
4,4,01-01 04:00,공주,0.189573,0.000000,0.002558,0.0,0.795
...,...,...,...,...,...,...,...,...
7723,4,11-18 19:00,공주,0.445498,0.791667,0.097187,0.0,0.368
7724,4,11-18 20:00,공주,0.426540,0.781667,0.104859,0.0,0.502
7725,4,11-18 21:00,공주,0.413902,0.735278,0.048593,0.0,0.484
7726,4,11-18 22:00,공주,0.404423,0.727222,0.058824,0.0,0.564


In [13]:
AWS_공주[AWS_공주['연도']==3].head()

,연도,일시,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
26280,3,01-01 00:00,공주,0.208531,0.109722,0.015345,0.0,0.785
26281,3,01-01 01:00,공주,0.192733,0.213056,0.017903,0.0,0.824
26282,3,01-01 02:00,공주,0.178515,0.000000,0.007673,0.0,0.831
26283,3,01-01 03:00,공주,0.173776,0.000000,0.010230,0.0,0.848
26284,3,01-01 04:00,공주,0.157978,0.327500,0.017903,0.0,0.865


## 함수 정리

In [14]:
import math

def incenter(x_lst, y_lst):  # 내심
    p1_x = x_lst[0];p1_y = y_lst[0]
    p2_x = x_lst[1];p2_y = y_lst[1]
    p3_x = x_lst[2];p3_y = y_lst[2]
    l12 = math.sqrt((p1_x - p2_x)**2 + (p1_y - p2_y)**2)
    l13 = math.sqrt((p1_x - p3_x)**2 + (p1_y - p3_y)**2)
    l23 = math.sqrt((p2_x - p3_x)**2 + (p2_y - p3_y)**2)
    X = (l12*p3_x + l13*p2_x + l23*p1_x) / (l12 + l13 + l23)
    Y = (l12*p3_y + l13*p2_y + l23*p1_y) / (l12 + l13 + l23)
    return (X,Y)

def centroid(x_lst, y_lst):   # 무게중심
    n = len(x_lst)
    X = 0;Y = 0
    X = sum(x_lst);Y = sum(y_lst)
    X /= n;Y /= n
    return (X,Y)

## aws 와의 거리 계산 함수 -> 단순 유클리드 거리가 아니라 gps상의 거리를 이용

def rad2deg(radians):
    degrees = radians * 180 / pi
    return degrees

def deg2rad(degrees):
    radians = degrees * pi / 180
    return radians

def getDistanceBetweenPointsNew(latitude1, longitude1, latitude2, longitude2, unit = 'miles'):
    theta = longitude1 - longitude2
    dist = 60 * 1.1515 * rad2deg(
        arccos(
            (sin(deg2rad(latitude1)) * sin(deg2rad(latitude2))) + 
            (cos(deg2rad(latitude1)) * cos(deg2rad(latitude2)) * cos(deg2rad(theta)))
        )
    )
    if unit == 'miles':
        return round(dist, 2)
    if unit == 'kilometers':
        return round(dist * 1.609344, 3)
def distance(p1, p2):  #거리
    p1_x = p1[0];p1_y = p1[1]
    p2_x = p2[0];p2_y = p2[1]
    D = getDistanceBetweenPointsNew(p1_x, p1_y, p2_x,p2_y, unit = 'kilometers')
    return D


In [15]:
rads = []
Near_AWS = []
Count_AWS = []
lctn_AWS = {}
for i in range(len(loc_PM.index)): # 17개 PM 관측소
    lat_PM = loc_PM['Latitude'][i]
    lon_PM = loc_PM['Longitude'][i]
    
    lats_AWS = [];lons_AWS = []
    
    near_AWS = ''
    rad = 0
    count_AWS = 0
    
    while len(near_AWS) == 0:
        # 변경
        rad += 5     # 반경을 조금씩 넓혀가며
        
        for j in range(len(loc_AWS.index)):
            lat_AWS = loc_AWS['Latitude'][j]
            lon_AWS = loc_AWS['Longitude'][j]
            if distance((lat_AWS, lon_AWS), (lat_PM, lon_PM)) < rad:
                count_AWS += 1
                lats_AWS.append(lat_AWS);lons_AWS.append(lon_AWS);
                near_AWS += str('/') + loc_AWS.index[j]
    
    rads.append(rad)
    Near_AWS.append(near_AWS[1:])
    Count_AWS.append(count_AWS)
    lctn_AWS[f'{loc_PM.index[i]}'] = [lats_AWS,lons_AWS]
    
loc_PM['Rad'] = rads
loc_PM['Near_AWS'] = Near_AWS
loc_PM['Count_AWS'] = Count_AWS
loc_PM

,Latitude,Longitude,Rad,Near_AWS,Count_AWS
Location,,,,,
공주,36.446951,127.119209,5,공주,1
노은동,36.368242,127.318498,10,계룡,1
논산,36.199217,127.087021,5,논산,1
대천2동,36.353148,126.589735,10,대천항,1
독곶리,36.987579,126.391672,5,대산,1
동문동,36.780158,126.455197,15,태안,1
모종동,36.782700,127.014610,20,세종전의/성거/예산/아산,4
문창동,36.317215,127.437825,5,오월드,1
성성동,36.840313,127.141777,5,성거,1


In [16]:
cent_lat = [];cent_lon=[];distance_PC = []   # 일단 무게중심, 위도경도만
for nm_PM in loc_PM.index:
    cent_point = centroid(lctn_AWS[nm_PM][0], lctn_AWS[nm_PM][1])
    distance_PC.append(distance(cent_point,(loc_PM.loc[nm_PM]['Latitude'],loc_PM.loc[nm_PM]['Longitude'])))
    cent_lat.append(cent_point[0]);cent_lon.append(cent_point[1])
    

loc_PM['Cent_lat'] = cent_lat
loc_PM['Cent_lon'] = cent_lon
loc_PM['PM-Cent_d'] = distance_PC  #뺄까말까

#변경 - 칼럼 추가
loc_PM['sub_lat'] = loc_PM['Latitude'] - loc_PM['Cent_lat']
loc_PM['sub_lon'] = loc_PM['Longitude'] - loc_PM['Cent_lon']

loc_PM

,Latitude,Longitude,Rad,Near_AWS,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d,sub_lat,sub_lon
Location,,,,,,,,,,
공주,36.446951,127.119209,5,공주,1,36.482800,127.13650,4.275,-0.035849,-0.017291
노은동,36.368242,127.318498,10,계룡,1,36.313200,127.24070,9.274,0.055042,0.077798
논산,36.199217,127.087021,5,논산,1,36.211600,127.10820,2.347,-0.012383,-0.021179
대천2동,36.353148,126.589735,10,대천항,1,36.324400,126.50210,8.475,0.028748,0.087635
독곶리,36.987579,126.391672,5,대산,1,37.010600,126.38810,2.579,-0.023021,0.003572
동문동,36.780158,126.455197,15,태안,1,36.758500,126.29640,14.347,0.021658,0.158797
모종동,36.782700,127.014610,20,세종전의/성거/예산/아산,4,36.789225,127.00910,0.876,-0.006525,0.005510
문창동,36.317215,127.437825,5,오월드,1,36.291300,127.39590,4.735,0.025915,0.041925
성성동,36.840313,127.141777,5,성거,1,36.878200,127.15610,4.401,-0.037887,-0.014323


## Train Dataset 만들기(AWS 이용)

In [17]:
df = pd.DataFrame()

In [18]:
'''
PM_loc = [PM_공주,PM_노은동, PM_논산,PM_대천2동,PM_독곶리,PM_동문동,PM_모종동, PM_문창동,PM_성성동,PM_신방동, PM_신흥동,
    PM_아름동,PM_예산군, PM_읍내동,PM_이원면, PM_정림동, PM_홍성읍]
'''

for i,nm_PM in enumerate(loc_PM.index):
    temp = PM_loc[i]
    #변경
    temp[['Rad','Count_AWS','sub_lat','sub_lon','PM-Cent_d']] = loc_PM.loc[nm_PM][['Rad','Count_AWS','sub_lat','sub_lon','PM-Cent_d']]
    connectaws = PM_loc[i].iloc[:,:2]
    listaws = list(loc_PM.iloc[i,3].split('/'))
    connectaws['기온(°C)'] = 0
    connectaws['풍향(deg)'] = 0
    connectaws['풍속(m/s)'] = 0
    connectaws['강수량(mm)'] = 0
    connectaws['습도(%)'] = 0
    for loc in listaws:
        a = pd.read_csv(f'./TRAIN_AWS/{loc}.csv',sep = ',|\n',encoding = 'UTF-8',error_bad_lines=False)
        connectaws['기온(°C)'] += a['기온(°C)']
        connectaws['풍향(deg)'] += a['풍향(deg)']
        connectaws['풍속(m/s)'] += a['풍속(m/s)']
        connectaws['강수량(mm)'] += a['강수량(mm)']
        connectaws['습도(%)'] += a['습도(%)']
    connectaws['기온(°C)'] /= len(listaws)
    connectaws['풍향(deg)']/= len(listaws)
    connectaws['풍속(m/s)'] /= len(listaws)
    connectaws['강수량(mm)']/= len(listaws)
    connectaws['습도(%)'] /= len(listaws)
    v = pd.merge(temp,connectaws,how='right',on=['연도','일시'])
    df = pd.concat([df,v], axis=0)
df.head()    

,연도,일시,측정소,PM2.5,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0,01-01 00:00,공주,0.056,5,1,-0.035849,-0.017291,4.275,0.173776,0.201944,0.023018,0.0,0.828
1,0,01-01 01:00,공주,0.060,5,1,-0.035849,-0.017291,4.275,0.176935,0.168611,0.030691,0.0,0.831
2,0,01-01 02:00,공주,0.068,5,1,-0.035849,-0.017291,4.275,0.180095,0.087222,0.033248,0.0,0.784
3,0,01-01 03:00,공주,0.060,5,1,-0.035849,-0.017291,4.275,0.178515,0.087222,0.025575,0.0,0.745
4,0,01-01 04:00,공주,0.068,5,1,-0.035849,-0.017291,4.275,0.164297,0.113889,0.020460,0.0,0.750


In [19]:
df.tail()

,연도,일시,측정소,PM2.5,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
35059,3,12-31 19:00,홍성읍,0.060,5,1,-0.030027,0.009987,3.456,0.273302,0.832222,0.086957,0.0,0.671
35060,3,12-31 20:00,홍성읍,0.052,5,1,-0.030027,0.009987,3.456,0.271722,0.831667,0.043478,0.0,0.692
35061,3,12-31 21:00,홍성읍,0.044,5,1,-0.030027,0.009987,3.456,0.268562,0.832500,0.066496,0.0,0.706
35062,3,12-31 22:00,홍성읍,0.052,5,1,-0.030027,0.009987,3.456,0.262243,0.866944,0.043478,0.0,0.725
35063,3,12-31 23:00,홍성읍,0.060,5,1,-0.030027,0.009987,3.456,0.257504,0.000000,0.000000,0.0,0.710


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 596088 entries, 0 to 35063
Data columns (total 14 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   연도         596088 non-null  int64  
 1   일시         596088 non-null  object 
 2   측정소        596088 non-null  object 
 3   PM2.5      580546 non-null  float64
 4   Rad        596088 non-null  int64  
 5   Count_AWS  596088 non-null  int64  
 6   sub_lat    596088 non-null  float64
 7   sub_lon    596088 non-null  float64
 8   PM-Cent_d  596088 non-null  float64
 9   기온(°C)     583200 non-null  float64
 10  풍향(deg)    583200 non-null  float64
 11  풍속(m/s)    583200 non-null  float64
 12  강수량(mm)    583200 non-null  float64
 13  습도(%)      583200 non-null  float64
dtypes: float64(9), int64(3), object(2)
memory usage: 68.2+ MB


In [21]:
## 달 ,날짜, 시간 구분해주기 -> 전부 int 정수형으로 
def time_spliter(df):
    df['달']=df['일시'].str.split("-").apply(lambda x: x[0]).apply(lambda x: int(x))
    
    
    df['날짜'] = df['일시'].str.split("-").apply(lambda x: x[1]).str.split(" ").apply(lambda x: x[0]).str.split(":").apply(lambda x: x[0]).apply(lambda x: int(x))
    df['시간'] = df['일시'].str.split("-").apply(lambda x: x[1]).str.split(" ").apply(lambda x: x[1]).str.split(":").apply(lambda x: x[0]).apply(lambda x: int(x))
    
    # 변경
    df['연간일자'] = df['달'] * 30 + df['날짜'] - 30
    
    #df.drop('일시',axis = 1,inplace = True)



## 주기성을 반영하기 위해서 달과 시간을 삼각함수에 집어 넣기 
# 참조
# https://dacon.io/competitions/official/235985/codeshare/7042?page=1&dtype=recent 

def sin_converter(df,col,a): ## 시간 변환 
    df[f"sin_{col}"] = df[col].apply(lambda x: math.sin(2*math.pi*x/a))

def cos_converter(df,col,a):
    df[f"cos_{col}"] = df[col].apply(lambda x: math.cos(2*math.pi*x/a))

def total_converter(df):
    sin_converter(df,'연간일자',365)
    cos_converter(df,'연간일자',365)

    sin_converter(df,'시간',24)
    cos_converter(df,'시간',24)
    
    sin_converter(df,'풍향(deg)',1)
    cos_converter(df,'풍향(deg)',1)


In [22]:
time_spliter(df)

In [23]:
total_converter(df)

In [24]:
#변경
PM = list(df['PM2.5'])
PM = [PM[0]] + PM[:len(PM)-1]

df['last_PM'] = PM
df

,연도,일시,측정소,PM2.5,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),...,날짜,시간,연간일자,sin_연간일자,cos_연간일자,sin_시간,cos_시간,sin_풍향(deg),cos_풍향(deg),last_PM
0,0,01-01 00:00,공주,0.056,5,1,-0.035849,-0.017291,4.275,0.173776,...,1,0,1,0.017213,0.999852,0.000000,1.000000,0.954761,0.297375,0.056
1,0,01-01 01:00,공주,0.060,5,1,-0.035849,-0.017291,4.275,0.176935,...,1,1,1,0.017213,0.999852,0.258819,0.965926,0.872069,0.489382,0.056
2,0,01-01 02:00,공주,0.068,5,1,-0.035849,-0.017291,4.275,0.180095,...,1,2,1,0.017213,0.999852,0.500000,0.866025,0.521010,0.853551,0.060
3,0,01-01 03:00,공주,0.060,5,1,-0.035849,-0.017291,4.275,0.178515,...,1,3,1,0.017213,0.999852,0.707107,0.707107,0.521010,0.853551,0.068
4,0,01-01 04:00,공주,0.068,5,1,-0.035849,-0.017291,4.275,0.164297,...,1,4,1,0.017213,0.999852,0.866025,0.500000,0.656059,0.754710,0.060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,3,12-31 19:00,홍성읍,0.060,5,1,-0.030027,0.009987,3.456,0.273302,...,31,19,361,-0.068802,0.997630,-0.965926,0.258819,-0.869495,0.493942,0.056
35060,3,12-31 20:00,홍성읍,0.052,5,1,-0.030027,0.009987,3.456,0.271722,...,31,20,361,-0.068802,0.997630,-0.866025,0.500000,-0.871214,0.490904,0.060
35061,3,12-31 21:00,홍성읍,0.044,5,1,-0.030027,0.009987,3.456,0.268562,...,31,21,361,-0.068802,0.997630,-0.707107,0.707107,-0.868632,0.495459,0.052
35062,3,12-31 22:00,홍성읍,0.052,5,1,-0.030027,0.009987,3.456,0.262243,...,31,22,361,-0.068802,0.997630,-0.500000,0.866025,-0.741976,0.670427,0.044


In [25]:
df

,연도,일시,측정소,PM2.5,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),...,날짜,시간,연간일자,sin_연간일자,cos_연간일자,sin_시간,cos_시간,sin_풍향(deg),cos_풍향(deg),last_PM
0,0,01-01 00:00,공주,0.056,5,1,-0.035849,-0.017291,4.275,0.173776,...,1,0,1,0.017213,0.999852,0.000000,1.000000,0.954761,0.297375,0.056
1,0,01-01 01:00,공주,0.060,5,1,-0.035849,-0.017291,4.275,0.176935,...,1,1,1,0.017213,0.999852,0.258819,0.965926,0.872069,0.489382,0.056
2,0,01-01 02:00,공주,0.068,5,1,-0.035849,-0.017291,4.275,0.180095,...,1,2,1,0.017213,0.999852,0.500000,0.866025,0.521010,0.853551,0.060
3,0,01-01 03:00,공주,0.060,5,1,-0.035849,-0.017291,4.275,0.178515,...,1,3,1,0.017213,0.999852,0.707107,0.707107,0.521010,0.853551,0.068
4,0,01-01 04:00,공주,0.068,5,1,-0.035849,-0.017291,4.275,0.164297,...,1,4,1,0.017213,0.999852,0.866025,0.500000,0.656059,0.754710,0.060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,3,12-31 19:00,홍성읍,0.060,5,1,-0.030027,0.009987,3.456,0.273302,...,31,19,361,-0.068802,0.997630,-0.965926,0.258819,-0.869495,0.493942,0.056
35060,3,12-31 20:00,홍성읍,0.052,5,1,-0.030027,0.009987,3.456,0.271722,...,31,20,361,-0.068802,0.997630,-0.866025,0.500000,-0.871214,0.490904,0.060
35061,3,12-31 21:00,홍성읍,0.044,5,1,-0.030027,0.009987,3.456,0.268562,...,31,21,361,-0.068802,0.997630,-0.707107,0.707107,-0.868632,0.495459,0.052
35062,3,12-31 22:00,홍성읍,0.052,5,1,-0.030027,0.009987,3.456,0.262243,...,31,22,361,-0.068802,0.997630,-0.500000,0.866025,-0.741976,0.670427,0.044


df.drop(['달','날짜','시간','연간일자','풍향(deg)'],axis=1, inplace=True)

In [26]:
df.columns

Index(['연도', '일시', '측정소', 'PM2.5', 'Rad', 'Count_AWS', 'sub_lat', 'sub_lon',
       'PM-Cent_d', '기온(°C)', '풍향(deg)', '풍속(m/s)', '강수량(mm)', '습도(%)', '달',
       '날짜', '시간', '연간일자', 'sin_연간일자', 'cos_연간일자', 'sin_시간', 'cos_시간',
       'sin_풍향(deg)', 'cos_풍향(deg)', 'last_PM'],
      dtype='object')

In [27]:
#변경
df = df[['연도','sin_연간일자', 'cos_연간일자', 'sin_시간', 'cos_시간', '측정소', 'Rad', 'Count_AWS', 'sub_lat', 'sub_lon',
       'PM-Cent_d', '기온(°C)', 'sin_풍향(deg)', 'cos_풍향(deg)' , '풍속(m/s)', '강수량(mm)', '습도(%)','last_PM','PM2.5']]
df.head()

,연도,sin_연간일자,cos_연간일자,sin_시간,cos_시간,측정소,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),sin_풍향(deg),cos_풍향(deg),풍속(m/s),강수량(mm),습도(%),last_PM,PM2.5
0,0,0.017213,0.999852,0.000000,1.000000,공주,5,1,-0.035849,-0.017291,4.275,0.173776,0.954761,0.297375,0.023018,0.0,0.828,0.056,0.056
1,0,0.017213,0.999852,0.258819,0.965926,공주,5,1,-0.035849,-0.017291,4.275,0.176935,0.872069,0.489382,0.030691,0.0,0.831,0.056,0.060
2,0,0.017213,0.999852,0.500000,0.866025,공주,5,1,-0.035849,-0.017291,4.275,0.180095,0.521010,0.853551,0.033248,0.0,0.784,0.060,0.068
3,0,0.017213,0.999852,0.707107,0.707107,공주,5,1,-0.035849,-0.017291,4.275,0.178515,0.521010,0.853551,0.025575,0.0,0.745,0.068,0.060
4,0,0.017213,0.999852,0.866025,0.500000,공주,5,1,-0.035849,-0.017291,4.275,0.164297,0.656059,0.754710,0.020460,0.0,0.750,0.060,0.068


In [28]:
loc = list(set(df['측정소']))
loc

['공주',
 '홍성읍',
 '논산',
 '문창동',
 '성성동',
 '노은동',
 '독곶리',
 '정림동',
 '동문동',
 '신방동',
 '읍내동',
 '이원면',
 '예산군',
 '아름동',
 '대천2동',
 '신흥동',
 '모종동']

In [29]:
from sklearn.preprocessing import LabelEncoder
observations = loc
encoder = LabelEncoder()
encoder.fit(observations)
df['측정소'] = encoder.transform(df['측정소'])

In [30]:
df.head()

,연도,sin_연간일자,cos_연간일자,sin_시간,cos_시간,측정소,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),sin_풍향(deg),cos_풍향(deg),풍속(m/s),강수량(mm),습도(%),last_PM,PM2.5
0,0,0.017213,0.999852,0.000000,1.000000,0,5,1,-0.035849,-0.017291,4.275,0.173776,0.954761,0.297375,0.023018,0.0,0.828,0.056,0.056
1,0,0.017213,0.999852,0.258819,0.965926,0,5,1,-0.035849,-0.017291,4.275,0.176935,0.872069,0.489382,0.030691,0.0,0.831,0.056,0.060
2,0,0.017213,0.999852,0.500000,0.866025,0,5,1,-0.035849,-0.017291,4.275,0.180095,0.521010,0.853551,0.033248,0.0,0.784,0.060,0.068
3,0,0.017213,0.999852,0.707107,0.707107,0,5,1,-0.035849,-0.017291,4.275,0.178515,0.521010,0.853551,0.025575,0.0,0.745,0.068,0.060
4,0,0.017213,0.999852,0.866025,0.500000,0,5,1,-0.035849,-0.017291,4.275,0.164297,0.656059,0.754710,0.020460,0.0,0.750,0.060,0.068


## TEST 데이터 영역

In [31]:
tdf = pd.DataFrame()

In [32]:
'''
TPM_loc = [TPM_공주,TPM_노은동, TPM_논산,TPM_대천2동,TPM_독곶리,TPM_동문동,TPM_모종동, TPM_문창동,TPM_성성동,TPM_신방동, TPM_신흥동,
    TPM_아름동,TPM_예산군, TPM_읍내동,TPM_이원면, TPM_정림동, TPM_홍성읍]
'''
TPM_loc[0]

,연도,일시,측정소,PM2.5
0,4,01-01 00:00,공주,0.060
1,4,01-01 01:00,공주,0.064
2,4,01-01 02:00,공주,0.072
3,4,01-01 03:00,공주,0.064
4,4,01-01 04:00,공주,0.056
...,...,...,...,...
7723,4,11-18 19:00,공주,0.052
7724,4,11-18 20:00,공주,0.056
7725,4,11-18 21:00,공주,0.036
7726,4,11-18 22:00,공주,0.044


In [33]:
for i,nm_PM in enumerate(loc_PM.index):
    temp = TPM_loc[i]
    temp[['Rad','Count_AWS','sub_lat','sub_lon','PM-Cent_d']] = loc_PM.loc[nm_PM][['Rad','Count_AWS','sub_lat','sub_lon','PM-Cent_d']]
    connectaws = TPM_loc[i].iloc[:,:2]
    listaws = list(loc_PM.iloc[i,3].split('/'))
    connectaws['기온(°C)'] = 0
    connectaws['풍향(deg)'] = 0
    connectaws['풍속(m/s)'] = 0
    connectaws['강수량(mm)'] = 0
    connectaws['습도(%)'] = 0
    err = 0
    for loc in listaws:
        a = pd.read_csv(f'./TEST_AWS/{loc}.csv',sep = ',|\n',encoding = 'UTF-8',error_bad_lines=False)
        a.fillna(method='ffill',inplace=True)
        connectaws['기온(°C)'] += a['기온(°C)']
        connectaws['풍향(deg)'] += a['풍향(deg)']
        connectaws['풍속(m/s)'] += a['풍속(m/s)']
        connectaws['강수량(mm)'] += a['강수량(mm)']
        connectaws['습도(%)'] += a['습도(%)']
    connectaws['기온(°C)'] /= (len(listaws))
    connectaws['풍향(deg)']/= len(listaws)
    connectaws['풍속(m/s)'] /= len(listaws)
    connectaws['강수량(mm)']/= len(listaws)
    connectaws['습도(%)'] /= len(listaws)
    v = pd.merge(temp,connectaws,how='right',on=['연도','일시'])
    tdf = pd.concat([tdf,v], axis=0)
tdf.head()    

,연도,일시,측정소,PM2.5,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,4,01-01 00:00,공주,0.060,5,1,-0.035849,-0.017291,4.275,0.244866,0.123333,0.038363,0.0,0.647
1,4,01-01 01:00,공주,0.064,5,1,-0.035849,-0.017291,4.275,0.232227,0.167778,0.033248,0.0,0.648
2,4,01-01 02:00,공주,0.072,5,1,-0.035849,-0.017291,4.275,0.206951,0.000000,0.002558,0.0,0.734
3,4,01-01 03:00,공주,0.064,5,1,-0.035849,-0.017291,4.275,0.199052,0.000000,0.002558,0.0,0.753
4,4,01-01 04:00,공주,0.056,5,1,-0.035849,-0.017291,4.275,0.189573,0.000000,0.002558,0.0,0.795


In [34]:
sample = tdf[tdf['PM2.5'].isnull()]
sample.head()

,연도,일시,측정소,PM2.5,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
48,4,01-03 00:00,공주,NaN,5,1,-0.035849,-0.017291,4.275,0.230648,0.145833,0.017903,0.0,0.707
49,4,01-03 01:00,공주,NaN,5,1,-0.035849,-0.017291,4.275,0.230648,0.145833,0.017903,0.0,0.707
50,4,01-03 02:00,공주,NaN,5,1,-0.035849,-0.017291,4.275,0.230648,0.145833,0.017903,0.0,0.707
51,4,01-03 03:00,공주,NaN,5,1,-0.035849,-0.017291,4.275,0.230648,0.145833,0.017903,0.0,0.707
52,4,01-03 04:00,공주,NaN,5,1,-0.035849,-0.017291,4.275,0.230648,0.145833,0.017903,0.0,0.707


In [35]:
time_spliter(tdf)

In [36]:
total_converter(tdf)

In [37]:
#변경
PM = list(tdf['PM2.5'])
PM = [PM[0]] + PM[:len(PM)-1]

tdf['last_PM'] = PM
tdf

,연도,일시,측정소,PM2.5,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),...,날짜,시간,연간일자,sin_연간일자,cos_연간일자,sin_시간,cos_시간,sin_풍향(deg),cos_풍향(deg),last_PM
0,4,01-01 00:00,공주,0.060,5,1,-0.035849,-0.017291,4.275,0.244866,...,1,0,1,0.017213,0.999852,0.000000,1.000000,0.699663,0.714473,0.060
1,4,01-01 01:00,공주,0.064,5,1,-0.035849,-0.017291,4.275,0.232227,...,1,1,1,0.017213,0.999852,0.258819,0.965926,0.869495,0.493942,0.060
2,4,01-01 02:00,공주,0.072,5,1,-0.035849,-0.017291,4.275,0.206951,...,1,2,1,0.017213,0.999852,0.500000,0.866025,0.000000,1.000000,0.064
3,4,01-01 03:00,공주,0.064,5,1,-0.035849,-0.017291,4.275,0.199052,...,1,3,1,0.017213,0.999852,0.707107,0.707107,0.000000,1.000000,0.072
4,4,01-01 04:00,공주,0.056,5,1,-0.035849,-0.017291,4.275,0.189573,...,1,4,1,0.017213,0.999852,0.866025,0.500000,0.000000,1.000000,0.064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,4,11-18 19:00,홍성읍,0.056,5,1,-0.030027,0.009987,3.456,0.404423,...,18,19,318,-0.723644,0.690173,-0.965926,0.258819,-0.865151,0.501511,0.068
7724,4,11-18 20:00,홍성읍,0.036,5,1,-0.030027,0.009987,3.456,0.396524,...,18,20,318,-0.723644,0.690173,-0.866025,0.500000,-0.874620,0.484810,0.056
7725,4,11-18 21:00,홍성읍,0.020,5,1,-0.030027,0.009987,3.456,0.393365,...,18,21,318,-0.723644,0.690173,-0.707107,0.707107,-0.852640,0.522499,0.036
7726,4,11-18 22:00,홍성읍,0.024,5,1,-0.030027,0.009987,3.456,0.387046,...,18,22,318,-0.723644,0.690173,-0.500000,0.866025,-0.936672,0.350207,0.020


tdf.drop(['달','날짜','시간'],axis=1, inplace=True)

tdf.columns

In [38]:
#변경
tdf = tdf[['연도','sin_연간일자', 'cos_연간일자', 'sin_시간', 'cos_시간', '측정소', 'Rad', 'Count_AWS', 'sub_lat', 'sub_lon',
       'PM-Cent_d', '기온(°C)', 'sin_풍향(deg)', 'cos_풍향(deg)' , '풍속(m/s)', '강수량(mm)', '습도(%)','last_PM','PM2.5']]
tdf.head()

,연도,sin_연간일자,cos_연간일자,sin_시간,cos_시간,측정소,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),sin_풍향(deg),cos_풍향(deg),풍속(m/s),강수량(mm),습도(%),last_PM,PM2.5
0,4,0.017213,0.999852,0.000000,1.000000,공주,5,1,-0.035849,-0.017291,4.275,0.244866,0.699663,0.714473,0.038363,0.0,0.647,0.060,0.060
1,4,0.017213,0.999852,0.258819,0.965926,공주,5,1,-0.035849,-0.017291,4.275,0.232227,0.869495,0.493942,0.033248,0.0,0.648,0.060,0.064
2,4,0.017213,0.999852,0.500000,0.866025,공주,5,1,-0.035849,-0.017291,4.275,0.206951,0.000000,1.000000,0.002558,0.0,0.734,0.064,0.072
3,4,0.017213,0.999852,0.707107,0.707107,공주,5,1,-0.035849,-0.017291,4.275,0.199052,0.000000,1.000000,0.002558,0.0,0.753,0.072,0.064
4,4,0.017213,0.999852,0.866025,0.500000,공주,5,1,-0.035849,-0.017291,4.275,0.189573,0.000000,1.000000,0.002558,0.0,0.795,0.064,0.056


In [39]:
tdf['측정소'] = encoder.transform(tdf['측정소'])

In [40]:
tdf.head()

,연도,sin_연간일자,cos_연간일자,sin_시간,cos_시간,측정소,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),sin_풍향(deg),cos_풍향(deg),풍속(m/s),강수량(mm),습도(%),last_PM,PM2.5
0,4,0.017213,0.999852,0.000000,1.000000,0,5,1,-0.035849,-0.017291,4.275,0.244866,0.699663,0.714473,0.038363,0.0,0.647,0.060,0.060
1,4,0.017213,0.999852,0.258819,0.965926,0,5,1,-0.035849,-0.017291,4.275,0.232227,0.869495,0.493942,0.033248,0.0,0.648,0.060,0.064
2,4,0.017213,0.999852,0.500000,0.866025,0,5,1,-0.035849,-0.017291,4.275,0.206951,0.000000,1.000000,0.002558,0.0,0.734,0.064,0.072
3,4,0.017213,0.999852,0.707107,0.707107,0,5,1,-0.035849,-0.017291,4.275,0.199052,0.000000,1.000000,0.002558,0.0,0.753,0.072,0.064
4,4,0.017213,0.999852,0.866025,0.500000,0,5,1,-0.035849,-0.017291,4.275,0.189573,0.000000,1.000000,0.002558,0.0,0.795,0.064,0.056


In [41]:
df.dropna(axis=0, inplace=True)

In [42]:
x_train = df.iloc[:, :-1]
y_train = df.iloc[:, -1]

In [43]:
write =tdf[tdf['PM2.5'].isnull()]
write.head()

,연도,sin_연간일자,cos_연간일자,sin_시간,cos_시간,측정소,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),sin_풍향(deg),cos_풍향(deg),풍속(m/s),강수량(mm),습도(%),last_PM,PM2.5
48,4,0.05162,0.998667,0.000000,1.000000,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,0.06,NaN
49,4,0.05162,0.998667,0.258819,0.965926,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,NaN,NaN
50,4,0.05162,0.998667,0.500000,0.866025,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,NaN,NaN
51,4,0.05162,0.998667,0.707107,0.707107,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,NaN,NaN
52,4,0.05162,0.998667,0.866025,0.500000,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,NaN,NaN


In [44]:
tdf.dropna(axis=0, inplace=True)

In [45]:
tdf.describe()

,연도,sin_연간일자,cos_연간일자,sin_시간,cos_시간,측정소,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),sin_풍향(deg),cos_풍향(deg),풍속(m/s),강수량(mm),습도(%),last_PM,PM2.5
count,51952.0,51952.000000,51952.000000,5.195200e+04,5.195200e+04,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000
mean,4.0,0.045060,-0.127685,-7.555530e-17,-2.094241e-02,8.000000,8.529412,1.294118,-0.005358,0.019165,5.025294,0.606036,-0.065273,0.099408,0.047808,0.002229,0.698211,0.069607,0.069833
std,0.0,0.735213,0.664189,7.144796e-01,6.993566e-01,4.899027,4.778892,0.748709,0.030766,0.050794,3.205369,0.160753,0.671720,0.731210,0.042176,0.015427,0.206325,0.056240,0.056532
min,4.0,-0.999991,-0.999667,-1.000000e+00,-1.000000e+00,0.000000,5.000000,1.000000,-0.064702,-0.057794,0.876000,0.142180,-1.000000,-1.000000,0.000000,0.000000,0.079000,0.004000,0.004000
25%,4.0,-0.780296,-0.766659,-7.071068e-01,-7.071068e-01,4.000000,5.000000,1.000000,-0.030027,-0.017291,2.994000,0.491311,-0.728969,-0.622515,0.017903,0.000000,0.553000,0.032000,0.032000
50%,4.0,0.192450,-0.183998,1.224647e-16,-1.836970e-16,8.000000,5.000000,1.000000,-0.006525,0.006189,4.401000,0.620853,0.000000,0.127065,0.038363,0.000000,0.725000,0.056000,0.056000
75%,4.0,0.763889,0.467359,7.071068e-01,7.071068e-01,12.000000,10.000000,1.000000,0.021658,0.041925,5.152000,0.740916,0.537300,0.880477,0.066496,0.000000,0.876000,0.092000,0.092000
max,4.0,0.999991,0.999852,1.000000e+00,1.000000e+00,16.000000,20.000000,4.000000,0.055042,0.158797,14.347000,0.938389,1.000000,1.000000,0.434783,0.647727,0.988000,0.480000,0.480000


In [46]:
x_test = tdf.iloc[:, :-1]
y_test = tdf.iloc[:, -1]

In [47]:
x_test.describe()

,연도,sin_연간일자,cos_연간일자,sin_시간,cos_시간,측정소,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),sin_풍향(deg),cos_풍향(deg),풍속(m/s),강수량(mm),습도(%),last_PM
count,51952.0,51952.000000,51952.000000,5.195200e+04,5.195200e+04,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000,51952.000000
mean,4.0,0.045060,-0.127685,-7.555530e-17,-2.094241e-02,8.000000,8.529412,1.294118,-0.005358,0.019165,5.025294,0.606036,-0.065273,0.099408,0.047808,0.002229,0.698211,0.069607
std,0.0,0.735213,0.664189,7.144796e-01,6.993566e-01,4.899027,4.778892,0.748709,0.030766,0.050794,3.205369,0.160753,0.671720,0.731210,0.042176,0.015427,0.206325,0.056240
min,4.0,-0.999991,-0.999667,-1.000000e+00,-1.000000e+00,0.000000,5.000000,1.000000,-0.064702,-0.057794,0.876000,0.142180,-1.000000,-1.000000,0.000000,0.000000,0.079000,0.004000
25%,4.0,-0.780296,-0.766659,-7.071068e-01,-7.071068e-01,4.000000,5.000000,1.000000,-0.030027,-0.017291,2.994000,0.491311,-0.728969,-0.622515,0.017903,0.000000,0.553000,0.032000
50%,4.0,0.192450,-0.183998,1.224647e-16,-1.836970e-16,8.000000,5.000000,1.000000,-0.006525,0.006189,4.401000,0.620853,0.000000,0.127065,0.038363,0.000000,0.725000,0.056000
75%,4.0,0.763889,0.467359,7.071068e-01,7.071068e-01,12.000000,10.000000,1.000000,0.021658,0.041925,5.152000,0.740916,0.537300,0.880477,0.066496,0.000000,0.876000,0.092000
max,4.0,0.999991,0.999852,1.000000e+00,1.000000e+00,16.000000,20.000000,4.000000,0.055042,0.158797,14.347000,0.938389,1.000000,1.000000,0.434783,0.647727,0.988000,0.480000


## 간단한 선형 모델 이용

In [48]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.linear_model import LinearRegression, Ridge, Lasso
lr_reg = LinearRegression()
lr_reg.fit(x_train, y_train)
ridge_reg = Ridge()
ridge_reg.fit(x_train, y_train)
lasso_reg = Lasso()
lasso_reg.fit(x_train, y_train)
models = [lr_reg, ridge_reg, lasso_reg]
for i,model in enumerate(models):
    pred = model.predict(x_test)
    print('{}의 mae:'.format(i), mae(pred,y_test))

0의 mae: 0.01315295953316872
1의 mae: 0.013153043377719812
2의 mae: 0.046817147099927854


In [49]:
result = write.iloc[:,:-1]

In [50]:
result = result.fillna(method='ffill')
result

,연도,sin_연간일자,cos_연간일자,sin_시간,cos_시간,측정소,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),sin_풍향(deg),cos_풍향(deg),풍속(m/s),강수량(mm),습도(%),last_PM
48,4,0.051620,0.998667,0.000000,1.000000,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,0.06
49,4,0.051620,0.998667,0.258819,0.965926,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,0.06
50,4,0.051620,0.998667,0.500000,0.866025,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,0.06
51,4,0.051620,0.998667,0.707107,0.707107,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,0.06
52,4,0.051620,0.998667,0.866025,0.500000,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7675,4,-0.746972,0.664855,-0.965926,0.258819,16,5,1,-0.030027,0.009987,3.456,0.461295,-0.978509,0.206204,0.025575,0.0,0.982,0.14
7676,4,-0.746972,0.664855,-0.866025,0.500000,16,5,1,-0.030027,0.009987,3.456,0.461295,-0.978509,0.206204,0.025575,0.0,0.982,0.14
7677,4,-0.746972,0.664855,-0.707107,0.707107,16,5,1,-0.030027,0.009987,3.456,0.461295,-0.978509,0.206204,0.025575,0.0,0.982,0.14
7678,4,-0.746972,0.664855,-0.500000,0.866025,16,5,1,-0.030027,0.009987,3.456,0.461295,-0.978509,0.206204,0.025575,0.0,0.982,0.14


In [51]:
ans = pd.read_csv('answer_sample.csv',error_bad_lines=False,sep = ',|\n|\t')

In [52]:
print(len(ans.index))

78336


In [53]:
result

,연도,sin_연간일자,cos_연간일자,sin_시간,cos_시간,측정소,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),sin_풍향(deg),cos_풍향(deg),풍속(m/s),강수량(mm),습도(%),last_PM
48,4,0.051620,0.998667,0.000000,1.000000,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,0.06
49,4,0.051620,0.998667,0.258819,0.965926,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,0.06
50,4,0.051620,0.998667,0.500000,0.866025,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,0.06
51,4,0.051620,0.998667,0.707107,0.707107,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,0.06
52,4,0.051620,0.998667,0.866025,0.500000,0,5,1,-0.035849,-0.017291,4.275,0.230648,0.793353,0.608761,0.017903,0.0,0.707,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7675,4,-0.746972,0.664855,-0.965926,0.258819,16,5,1,-0.030027,0.009987,3.456,0.461295,-0.978509,0.206204,0.025575,0.0,0.982,0.14
7676,4,-0.746972,0.664855,-0.866025,0.500000,16,5,1,-0.030027,0.009987,3.456,0.461295,-0.978509,0.206204,0.025575,0.0,0.982,0.14
7677,4,-0.746972,0.664855,-0.707107,0.707107,16,5,1,-0.030027,0.009987,3.456,0.461295,-0.978509,0.206204,0.025575,0.0,0.982,0.14
7678,4,-0.746972,0.664855,-0.500000,0.866025,16,5,1,-0.030027,0.009987,3.456,0.461295,-0.978509,0.206204,0.025575,0.0,0.982,0.14


In [54]:
answer = ridge_reg.predict(result)

In [55]:
result.describe()

,연도,sin_연간일자,cos_연간일자,sin_시간,cos_시간,측정소,Rad,Count_AWS,sub_lat,sub_lon,PM-Cent_d,기온(°C),sin_풍향(deg),cos_풍향(deg),풍속(m/s),강수량(mm),습도(%),last_PM
count,78336.0,78336.000000,78336.000000,7.833600e+04,7.833600e+04,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000
mean,4.0,0.051467,-0.143439,-7.400849e-17,-7.401062e-17,8.000000,8.529412,1.294118,-0.005358,0.019165,5.025294,0.565545,0.095862,0.193562,0.035310,0.001660,0.801399,0.082474
std,0.0,0.737760,0.657647,7.071113e-01,7.071113e-01,4.899011,4.778877,0.748706,0.030765,0.050794,3.205359,0.156820,0.587315,0.780011,0.040269,0.012370,0.145937,0.067375
min,4.0,-0.999769,-0.999963,-1.000000e+00,-1.000000e+00,0.000000,5.000000,1.000000,-0.064702,-0.057794,0.876000,0.178515,-1.000000,-0.999507,0.000000,0.000000,0.293000,0.004000
25%,4.0,-0.776107,-0.780274,-7.071068e-01,-7.071068e-01,4.000000,5.000000,1.000000,-0.030027,-0.017291,2.994000,0.451817,-0.177085,-0.639261,0.007673,0.000000,0.703000,0.036000
50%,4.0,0.200844,-0.217723,6.123234e-17,-6.123234e-17,8.000000,5.000000,1.000000,-0.006525,0.006189,4.401000,0.579779,0.000000,0.399549,0.023018,0.000000,0.844000,0.064000
75%,4.0,0.759574,0.440506,7.071068e-01,7.071068e-01,12.000000,10.000000,1.000000,0.021658,0.041925,5.152000,0.696090,0.552846,1.000000,0.048593,0.000000,0.919000,0.108000
max,4.0,0.999991,0.998667,1.000000e+00,1.000000e+00,16.000000,20.000000,4.000000,0.055042,0.158797,14.347000,0.829384,0.999999,1.000000,0.414322,0.193182,0.988000,0.400000


In [56]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78336 entries, 48 to 7679
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   연도           78336 non-null  int64  
 1   sin_연간일자     78336 non-null  float64
 2   cos_연간일자     78336 non-null  float64
 3   sin_시간       78336 non-null  float64
 4   cos_시간       78336 non-null  float64
 5   측정소          78336 non-null  int32  
 6   Rad          78336 non-null  int64  
 7   Count_AWS    78336 non-null  int64  
 8   sub_lat      78336 non-null  float64
 9   sub_lon      78336 non-null  float64
 10  PM-Cent_d    78336 non-null  float64
 11  기온(°C)       78336 non-null  float64
 12  sin_풍향(deg)  78336 non-null  float64
 13  cos_풍향(deg)  78336 non-null  float64
 14  풍속(m/s)      78336 non-null  float64
 15  강수량(mm)      78336 non-null  float64
 16  습도(%)        78336 non-null  float64
 17  last_PM      78336 non-null  float64
dtypes: float64(14), int32(1), int64(3)
memory usag

In [57]:
ans.head()

,연도,일시,측정소,PM2.5
0,4,01-03 00:00,공주,NaN
1,4,01-03 01:00,공주,NaN
2,4,01-03 02:00,공주,NaN
3,4,01-03 03:00,공주,NaN
4,4,01-03 04:00,공주,NaN


In [58]:
ans['PM2.5']= answer

In [59]:
ans.head()

,연도,일시,측정소,PM2.5
0,4,01-03 00:00,공주,0.063709
1,4,01-03 01:00,공주,0.063390
2,4,01-03 02:00,공주,0.063125
3,4,01-03 03:00,공주,0.062934
4,4,01-03 04:00,공주,0.062828


In [60]:
ans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78336 entries, 0 to 78335
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연도      78336 non-null  int64  
 1   일시      78336 non-null  object 
 2   측정소     78336 non-null  object 
 3   PM2.5   78336 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.4+ MB


In [61]:
ans.to_csv(path_or_buf='sss.csv', sep = ',', index = False)

## XGB 이용

In [62]:
def evaluate_regr(y,pred):
    mae_val = mae(y,pred)
    print(f'MAE:{mae_val}')

In [63]:
def get_model_predict(model,x_train,x_test,y_train,y_test,is_expm1=False):
    model.fit(x_train,y_train)
    pred = model.predict(x_test)
    if is_expm1:
        y_test = np.expm1(y_test)
        pred = np.expm1(pred)
    evaluate_regr(y_test,pred)

In [64]:
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor 
xgb_reg = XGBRegressor(n_estimators=500)
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)
get_model_predict(xgb_reg,x_train,x_test,y_train_log,y_test_log,is_expm1=True)

MAE:0.013819850513820217


In [65]:
log_answer= xgb_reg.predict(result)

In [66]:
answer = np.expm1(log_answer)

In [67]:
ans['PM2.5']= answer

In [68]:
ans.head()

,연도,일시,측정소,PM2.5
0,4,01-03 00:00,공주,0.066112
1,4,01-03 01:00,공주,0.066104
2,4,01-03 02:00,공주,0.064971
3,4,01-03 03:00,공주,0.064803
4,4,01-03 04:00,공주,0.065511


In [69]:
ans.to_csv(path_or_buf='XGB.csv')